<center><h1>Cross Validation</h1></center>

## 1. Introducción

La validación de entrenamiento/prueba es una técnica simple para probar la precisión de un modelo de aprendizaje automático en datos nuevos en los que el modelo no fue entrenado. Ahora, nos centraremos en técnicas más sólidas.

Para empezar, nos centraremos en la técnica de **validación por exclusión (holdout validation)**, que implica:

- dividir el conjunto de datos completo en 2 particiones:
    - un conjunto de entrenamiento
    - un conjunto de prueba
- entrenar el modelo en el conjunto de entrenamiento,
- usar el modelo entrenado para predecir etiquetas en el conjunto de prueba,
- calcular una métrica de error para comprender la efectividad del modelo,
- cambiar los conjuntos de entrenamiento y prueba y repita,
- promediar los errores.

En la validación por exclusión, generalmente usamos una división 50/50 en lugar de la división 75/25 de la validación de entrenamiento/prueba. De esta forma, eliminamos el número de observaciones como fuente potencial de variación en el rendimiento de nuestro modelo.

<img src="figs/holdout_validation.png" width="600" height="400" />


Comencemos por dividir el conjunto de datos en 2 mitades casi equivalentes.

Cuando divida el conjunto de datos, no olvide configurar una copia usando `.copy()` para asegurarse de no obtener resultados inesperados más adelante. Si ejecuta el código localmente en Jupyter Notebook o Jupyter Lab sin `.copy()`, notará lo que se conoce como Advertencia de configuración con copia. Esto no evitará que su código se ejecute correctamente, pero le permite saber que cualquier operación que esté haciendo está tratando de establecerse en una copia de un segmento de un dataframe. Para asegurarse de que no vea esta advertencia, asegúrese de incluir `.copy()` cada vez que realice operaciones en un dataframe.

### Ejercicio

- Utilice la función `numpy.random.permutation()` para aleatorizar el orden de las filas en `dc_listings`.
- Seleccione las primeras 1862 filas y asígnelas a `split_one`.
- Seleccione las 1861 filas restantes y asígnelas a `split_two`.

In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [2]:
dc_listings = pd.read_csv("dc_airbnb.csv")

In [3]:
dc_listings['price'] = dc_listings['price'].str.replace('[\$,]', '', regex=True).astype('float')

print(len(dc_listings) / 2, '\n')

1861.5 



In [4]:
dc_listings = dc_listings.iloc[np.random.default_rng(2021).permutation(len(dc_listings))]

In [5]:
split_one = dc_listings.iloc[:1862].copy()
split_two = dc_listings.iloc[1862:].copy()

## 2. Holdout Validation
Ahora que hemos dividido nuestro conjunto de datos en 2 dataframes, hagamos lo siguiente:

- entrenar un modelo de k-vecinos más cercanos en la primera mitad,
- probar este modelo en la segunda mitad,
- entrenar un modelo de k-vecinos más cercanos en la segunda mitad,
- probar este modelo en la primera mitad.

### Ejercicio

- Entrene un modelo de k-vecinos más cercanos utilizando el algoritmo predeterminado (`auto`) y el número predeterminado de vecinos (`5`) que:
     - Utiliza la columna `accommodates` de `train_one` para entrenar y
     - Lo prueba en `test_one`.
- Asigne el valor RMSE resultante a `iteration_one_rmse`.
- Entrene un modelo de k-vecinos más cercanos utilizando el algoritmo predeterminado (`auto`) y el número predeterminado de vecinos (`5`) que:
     - Utiliza la columna `accommodates` de `test_two` para entrenar y
     - Lo prueba en `test_two`.
- Asigne el valor RMSE resultante a `iteration_two_rmse`.
- Use `numpy.mean()` para calcular el promedio de los 2 valores RMSE y asigne a `avg_rmse`.

In [6]:
train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one

In [7]:
knn1 = KNeighborsRegressor()
knn1.fit(train_one[['accommodates']], train_one[['price']])
predictions1 = knn1.predict(test_one[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(predictions1, test_one['price']))

In [8]:
knn2 = KNeighborsRegressor()
knn2.fit(train_two[['accommodates']], train_two['price'])
predictions2 = knn2.predict(test_two[['accommodates']])
iteration_two_rmse = np.sqrt(mean_squared_error(predictions2, test_two['price']))

In [9]:
print(iteration_one_rmse)
print(iteration_two_rmse)

128.70692056945418
125.33395047116689


In [10]:
avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])
print(avg_rmse)

127.02043552031054


## 3. Validación cruzada K-Fold

Si promediamos los dos valores RMSE del último paso, obtenemos un valor RMSE de aproximadamente 127.02. La validación por exclusión (holdout validation) es en realidad un ejemplo específico de una clase más grande de técnicas de validación llamada validación cruzada k-fold. 

Si bien la validación de exclusión es mejor que la validación de entrenamiento/prueba porque el modelo no está sesgado repetidamente hacia un subconjunto específico de datos, ambos modelos entrenados solo usan la mitad de los datos disponibles. La validación cruzada K-fold, por otro lado, aprovecha una mayor proporción de los datos durante el entrenamiento mientras sigue rotando a través de diferentes subconjuntos de datos para evitar los problemas de validación de entrenamiento/prueba.

Aquí está el algoritmo de la validación cruzada de k-fold:

- dividir el conjunto de datos completo en `k` particiones de igual longitud.
    - seleccionando las `k-1` particiones  como el conjunto de entrenamiento y
    - seleccionando la partición restante como el conjunto de prueba
- entrenar el modelo en el conjunto de entrenamiento.
- usar el modelo entrenado para predecir etiquetas en el conjunto de prueba.
- calcular la métrica de error del conjunto prueba.
- repetir todos los pasos anteriores `k-1` veces, hasta que cada partición se haya utilizado como conjunto de prueba para una iteración.
- calcular la media de los valores de error `k`.

La validación de exclusión es esencialmente una versión de la validación cruzada k-fold cuando `k` es igual a `2`. En general, los pliegues (número de subconjuntos) de `5` o `10` se utilizan para la validación cruzada k-fold. Aquí hay un diagrama que describe cada iteración de la validación cruzada de 5 subconjuntos:

<img src="figs/kfold_cross_validation.png" width="800" height="600" />

A medida que aumenta el número de pliegues, el número de observaciones en cada pliegue (subconjunto) disminuye y la varianza de los errores entre cada pliegue aumenta. 

Comencemos dividiendo manualmente el conjunto de datos en 5 pliegues. En lugar de dividir en 5 dataframes, agreguemos una columna que especifique a qué subconjunto pertenece la fila. De esta manera, podemos seleccionar fácilmente nuestro conjunto de entrenamiento y nuestro conjunto de prueba.

### Ejercicio

- Agregue una nueva columna a `dc_listings` llamada `fold` que contiene el número de subconjunto al que pertenece cada fila:
- El pliegue `1` debe tener filas desde el índice `0` hasta `745`, sin incluir `745`.
- El pliegue `2` debe tener filas desde el índice `745` hasta `1490`, sin incluir `1490`.
- El pliegue `3` debe tener filas desde el índice `1490` hasta `2234`, sin incluir `2234`.
- El pliegue `4` debe tener filas desde el índice `2234` hasta `2978`, sin incluir `2978`.
- El pliegue `5` debe tener filas desde el índice `2978` hasta `3723`, sin incluir `3723`.
- Asegúrate de que el tipo de `fold` sea un tipo flotante.
- Cuente los valores únicos para la columna `fold` para confirmar que cada pliegue tiene aproximadamente la misma cantidad de elementos.
- Muestra el número de valores faltantes (missing values) en la columna `fold` para confirmar que no nos falta ninguna fila.

In [11]:
dc_listings.loc[dc_listings.index[:745], 'fold'] = 1.0

In [12]:
dc_listings['fold']

604     1.0
2493    1.0
904     1.0
1935    1.0
544     1.0
       ... 
2127    NaN
3388    NaN
1769    NaN
1333    NaN
3412    NaN
Name: fold, Length: 3723, dtype: float64

In [13]:
dc_listings.loc[dc_listings.index[745:1490], 'fold'] = 2
dc_listings.loc[dc_listings.index[1490:2234], 'fold'] = 3
dc_listings.loc[dc_listings.index[2234:2978], 'fold'] = 4
dc_listings.loc[dc_listings.index[2978:], 'fold'] = 5

In [14]:
dc_listings['fold'].value_counts().sort_index()

1.0    745
2.0    745
3.0    744
4.0    744
5.0    745
Name: fold, dtype: int64

In [15]:
# c_listings['fold'] = 1.0
# dc_listings.fold.iloc[745:1490] = 2
# dc_listings.fold.iloc[1490:2234] = 3
# dc_listings.fold.iloc[2234:2978] = 4
# dc_listings.fold.iloc[2978:] = 5

In [16]:
print(dc_listings['fold'].isnull().sum())

0


## 4. Primera iteración

Comencemos por realizar la primera iteración de la validación cruzada de k-fold en un modelo univariado.

### Ejercicio

- Entrene un modelo de k-vecinos más cercanos usando la columna `accommodates` como la única característica de los pliegues `2` a `5` como conjunto de entrenamiento.
- Use el modelo para hacer predicciones en el conjunto de prueba (columna `accommodates` del subconjunto `1`) y asigne las etiquetas pronosticadas a `labels`.
- Calcule el valor RMSE comparando la columna `price` con las etiquetas pronosticadas.
- Asigne el valor RMSE a `iteration_one_rmse`.

In [17]:
print(dc_listings[dc_listings['fold'] != 1]['fold'].unique())

[2. 3. 4. 5.]


In [18]:
def select_kfold(k, mode='exclude'):
    if mode == 'exclude':
        return dc_listings[dc_listings['fold'] != k].copy()
    elif mode == 'include':
        return dc_listings[dc_listings['fold'] == k].copy()
    raise Exception("Write mode parameter as 'exclude' or 'include'")

In [19]:
training_df = select_kfold(1, mode='exclude')
test_df = select_kfold(1, mode='include')

In [20]:
knn = KNeighborsRegressor()
knn.fit(training_df[['accommodates']], training_df['price'])
labels = knn.predict(test_df[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(labels, test_df['price']))

In [21]:
print(iteration_one_rmse)

111.45677104635922


## 5. Función para entrenar modelos

Desde la primera iteración, logramos un valor RMSE de aproximadamente **111**. Calculemos los valores RMSE para las iteraciones restantes. Para facilitar el proceso de iteración, coloquemos el código que escribimos anteriormente en una función.


### Ejercicio

- Escriba una función llamada `train_and_validate` que tome un dataframe como primer parámetro (`df`) y una lista de valores de subconjuntos (`1` a `5` en nuestro caso) como segundo parámetro (`folds`). Esta función debería:
    - Entrenar modelos `n` (donde `n` es el número de subconjuntos) y realizar una validación cruzada k-fold (usando `n` subconjuntos). Utilice el valor `k` predeterminado para la clase `KNeighborsRegressor`.
    - Devuelva una lista de valores RMSE, donde el primer elemento es el RMSE cuando el subconjunto `1` fue el conjunto de prueba, el segundo elemento es el RMSE para cuando el subconjunto `2` fue el conjunto de prueba, y así sucesivamente.
- Use la función `train_and_validate` para devolver la lista de valores RMSE para el dataframe `dc_listings` y asigne a `rmses`.
- Calcular la media de estos valores y asignar a `avg_rmse`.
- Mostrar tanto `rmses` como `avg_rmse`.

In [ ]:
def train_and_validate(df, folds):
    rmses = []
    
    for n in folds:
        training_df = df[df['fold'] != n].copy()
        test_df = df[df['fold'] == n].copy()

        knn = KNeighborsRegressor() 
        knn.fit(training_df[['accommodates']], training_df['price'])

        labels = knn.predict(test_df[['accommodates']])
        rmses.append(np.sqrt(mean_squared_error(labels, test_df['price'])))
    return rmses

In [ ]:
rmses = train_and_validate(dc_listings, folds=[1, 2, 3, 4, 5])
avg_rmse = np.mean(rmses)

In [ ]:
print(rmses)
print(avg_rmse)

## 6. Performing K-Fold Cross Validation Using Scikit-Learn

While the average RMSE value was approximately 135, the RMSE values ranged from 102 to 159+. This large amount of variability between the RMSE values means that we're either using a poor model or a poor evaluation criteria (or a bit of both!). By implementing your own k-fold cross-validation function, you hopefully acquired a good understanding of the inner workings of the technique. The function we wrote, however, has many limitations. If we want to now change the number of folds we want to use, we need to make the function more general so it can also handle randomizing the ordering of the rows in the dataframe and splitting into folds.

In machine learning, we're interested in building a good model and accurately understanding how well it will perform. To build a better k-nearest neighbors model, we can change the features it uses or tweak the number of neighbors (a hyperparameter). To accurately understand a model's performance, we can perform k-fold cross validation and select the proper number of folds. We've learned how scikit-learn makes it easy for us to quickly experiment with these different knobs when it comes to building a better model. Let's now dive into how we can use scikit-learn to handle cross-validation as well.

First, we instantiate an instance of the [KFold class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) from `sklearn.model_selection`:

```python
from sklearn.model_selection import KFold
kf = KFold(n_splits, shuffle=False, random_state=None)
```

where:

- `n_splits` is the number of folds you want to use,
- `shuffle` is used to toggle shuffling of the ordering of the observations in the dataset,
- `random_state` is used to specify the random seed value if `shuffle` is set to `True`.

You'll notice here that no parameters depend on the data set at all. This is because the KFold class returns an iterator object which we use in conjunction with the [cross_val_score()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function, also from `sklearn.model_selection`. Together, these 2 functions allow us to compactly train and test using k-fold cross validation:

Here are the relevant parameters for the `cross_val_score` function:

```python
from sklearn.model_selection import cross_val_score
cross_val_score(estimator, X, Y, scoring=None, cv=None)
```

where:

- `estimator` is a sklearn model that implements the `fit` method (e.g. instance of KNeighborsRegressor),
- `X` is the list or 2D array containing the features you want to train on,
- `y` is a list containing the values you want to predict (target column),
- `scoring` is a string describing the scoring criteria (list of accepted values here).
- `cv` describes the number of folds. Here are some examples of accepted values:
    - an instance of the `KFold` class,
    - an integer representing the number of folds.

Depending on the scoring criteria you specify, a single total value is returned for each fold. Here's the general workflow for performing k-fold cross-validation using the classes we just described:

- instantiate the scikit-learn model class you want to fit,
- instantiate the `KFold` class and using the parameters to specify the k-fold cross-validation attributes you want,
- use the `cross_val_score()` function to return the scoring metric you're interested in.


### Exercise

- Create a new instance of the `KFold` class with the following properties:
    - `5` folds,
    - shuffle set to `True`,
    - random seed set to `1` (so we can answer check using the same seed),
    - assigned to the variable `kf`.
- Create a new instance of the `KNeighborsRegressor` class and assign to `knn`.
- Use the `cross_val_score()` function to perform k-fold cross-validation:
    - using the KNeighborsRegressor instance `knn`,
    - using the `accommodates` column for training,
    - using the `price` column as the target column,
    - using the string `neg_mean_squared_error` as the value of the `scoring` parameter,
    - using `kf` as the value of the `cv` parameter
    - returning an array of MSE values (one value for each fold).
- Assign the resulting list of MSE values to `mses`. Then, take the absolute value followed by the square root of each MSE value. Then, calculate the average of the resulting RMSE values and assign to `avg_rmse`.


In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
knn = KNeighborsRegressor()

In [ ]:
mses = cross_val_score(estimator=knn, 
                       X=dc_listings[['accommodates']], 
                       y=dc_listings['price'], 
                       scoring='neg_mean_squared_error', 
                       cv=kf)

In [ ]:
avg_rmse = np.sqrt(np.abs(mses)).mean()
print(avg_rmse)

## 7. Exploring Different K Values
Choosing the right `k` value when performing k-fold cross validation is more of an art and less of a science. As we discussed earlier in the lesson, a `k` value of `2` is really just holdout validation. On the other end, setting `k` equal to `n` (the number of observations in the data set) is known as **leave-one-out cross validation**, or LOOCV for short. Through lots of trial and error, data scientists have converged on `10` as the standard k value.

In the following code block, we display the results of varying `k` from `3` to `23`. For each `k` value, we calculate and display the average RMSE value across all of the folds and the standard deviation of the RMSE values. Across the many different `k` values, it seems like the average RMSE value is around `129`. You'll notice that the standard deviation of the RMSE increases from approximately `8` to over `40` as we increase the number of folds.



In [ ]:
num_folds = [3, 5, 7, 9, 10, 11, 13, 15, 17, 19, 21, 23]

for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates"]], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print(f'{fold} folds: avg RMSE: {avg_rmse}, std RMSE: {std_rmse}')

## 8. Bias-Variance Tradeoff

So far, we've been working under the assumption that a lower RMSE always means that a model is more accurate. This isn't the complete picture, unfortunately. A model has two sources of error, **bias** and **variance**.

Bias describes error that results in bad assumptions about the learning algorithm. For example, assuming that only one feature, like a car's weight, relates to a car's fuel efficiency will lead you to fit a simple, univariate regression model that will result in high bias. The error rate will be high since a car's fuel efficiency is affected by many other factors besides just its weight.

Variance describes error that occurs because of the variability of a model's predicted values. If we were given a dataset with 1000 features on each car and used every single feature to train an incredibly complicated multivariate regression model, we will have low bias but high variance. In an ideal world, we want low bias and low variance but in reality, there's always a tradeoff.

The standard deviation of the RMSE values can be a proxy for a model's **variance** while the average RMSE is a proxy for a model's **bias**. Bias and variance are the 2 observable sources of error in a model that we can indirectly control.


<img src="figs/bias_variance.png" width="600" height="400" />

While k-nearest neighbors can make predictions, it isn't a mathematical model. A mathematical model is usually an equation that can exist without the original data, which isn't true with k-nearest neighbors. In the next two courses, we'll learn about a mathematical model called linear regression. We'll explore the bias-variance tradeoff in greater depth in these next 2 courses because of its importance when working with mathematical models in particular.

## 9. Next steps

In this lesson, we explored more robust cross validation techniques like holdout validation and k-fold cross-validation. Next in this course is a guided project where you can practice what you've learned in this course on a different data set.
